In [87]:
import numpy.linalg as la
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

In [88]:
#Visualization of 2 Group Variables

Sigma_a1, Sigma_a2, Sigma_1to2, Sigma_2to1, phi_1, phi_2, k, Xi_1, Xi_2, nu, Sigma_f1, Sigma_f2 = S('Sigma_a1, Sigma_a2, Sigma_1to2, Sigma_2to1, phi_1, phi_2, k, Xi_1, Xi_2, nu, Sigma_f1, Sigma_f2')

flux = Matrix([[phi_1],[phi_2]])
Xi = Matrix([[Xi_1],[Xi_2]])
Sigma_f = Matrix([Sigma_f1, Sigma_f2]).T

Absorption = Matrix([[Sigma_a1, 0],[0, Sigma_a2]]) #@ flux
Outscattering = Matrix([[Sigma_1to2, 0],[0,Sigma_2to1]]) #@ flux

Fission = nu * Xi @ Sigma_f #@ flux
Inscattering = Matrix([[0, Sigma_2to1],[Sigma_1to2, 0]]) #@ flux

Migration = (Absorption + Outscattering - Inscattering)

eq1 = Eq(Absorption @ flux + Outscattering @ flux - Inscattering @ flux, 1/k * Fission @ flux)
eq1

Eq(Matrix([
[ Sigma_1to2*phi_1 - Sigma_2to1*phi_2 + Sigma_a1*phi_1],
[-Sigma_1to2*phi_1 + Sigma_2to1*phi_2 + Sigma_a2*phi_2]]), Matrix([
[Sigma_f1*Xi_1*nu*phi_1/k + Sigma_f2*Xi_1*nu*phi_2/k],
[Sigma_f1*Xi_2*nu*phi_1/k + Sigma_f2*Xi_2*nu*phi_2/k]]))

In [89]:
#Power Iteration

def power_iteration(A, steps):
    phi_a = np.random.rand(A.shape[1])
    
    for _ in range(steps):
        phi_b = np.dot(A, phi_a)
        phi_norm = phi_a.dot(phi_a)**(1/2)
        phi_a = phi_b / phi_norm
        
        k = np.dot(np.dot(A, phi_a).T, phi_a) / (phi_a.T.dot(phi_a))
        
    return phi_a, k

In [90]:
#Reading

with open('NPRE 247_CP 2_input', 'r') as file:
    input = file.read()
input_split = input.split('\n')

every = list()

for i in (input_split):
    a = i.split('=')
    b = a[1].split(',')
    for j in range(len(b)):
        b[j] = float(b[j])
    every.append(b)

#Initializing Arrays (2 Group)
Abs2 = Matrix(every[0])
Fiss2 = Matrix(every[1])
Xi2 = Matrix(every[2])
Scat_In2 = Matrix([[0,0],[every[4][0],0]])
Scat_Out2 = Matrix([[every[4][0],0],[0,0]])

#Initializing Array (8 group)
Abs8 = Matrix(every[5])
Fiss8 = Matrix(every[6])
Xi8 = Matrix(every[7])

Scat_In8 = zeros(8)
for i in range(shape(Scat_In8)[0]):
    for j in range(shape(Scat_In8)[0]):
        if(i != j):
            Scat_In8[i,j] = every[i+8][j]

Scat_Out8 = zeros(8)
for i in range(shape(Scat_Out8)[0]):
    for j in range(shape(Scat_Out8)[0]):
        if (i != j):
            Scat_Out8[i,i] += every[j+8][i]
            
g = open('NPRE 247_CP 2_output', 'a')
g.write('Original Data:\n')
g.write(input)
g.write('\n' + '\n' + '\n')
g.close()

# Group 2

In [91]:
#2 Group, Defining

Absorption2 = Abs2
Absorption2 = Absorption2 @ Matrix([1,1]).T
Absorption2[1,0] = Absorption2[0,1] = 0

Migration2 = Absorption2 + Scat_Out2 - Scat_In2
Fission2 = Xi2 @ Fiss2.T

function2 = Migration2**(-1) @ Fission2

In [92]:
#2 Group, Migration Matrix

Migration2

Matrix([
[ 0.0294,      0],
[-0.0202, 0.0932]])

In [93]:
#2 Group, Fission Matrix

Fission2

Matrix([
[0.0046, 0.1139],
[     0,      0]])

In [94]:
#2 Group, Eigenvalues & Eigenvectors

Fun2Eigen = function2.eigenvects()

g = open('NPRE 247_CP 2_output', 'a')
g.write('Two-Group Eigen:\n')
for i in Fun2Eigen:
    g.write(str(i))
    g.write('\n' + '\n')
g.write('\n')
g.close()

Fun2Eigen

[(-2.12911042727575e-17,
  1,
  [Matrix([
   [  0.999185469505495],
   [-0.0403534078992562]])]),
 (0.996138798867186,
  1,
  [Matrix([
   [ 1.00965294385214],
   [0.218830359075249]])])]

In [95]:
#2 Group, Power Iteration, Eigenvalue

Max_eigen2 = power_iteration(function2, 2)
Max_eigenvalue2 = Max_eigen2[1]
Max_eigenvalue2

0.996138798867186

In [96]:
#2 Group, Power Iteration, Eigenvector

Max_eigenvector2 = Max_eigen2[0]
Max_eigenvector2 = Matrix(Max_eigenvector2)
Max_eigenvector2

Matrix([
[0.973535084915876],
[0.211002239434557]])

# Group 8

In [97]:
#8 group defining

Absorption8 = Abs8
Absorption8 = Matrix.diag(Absorption8[0], Absorption8[1], Absorption8[2], Absorption8[3], Absorption8[4], Absorption8[5], Absorption8[6], Absorption8[7])

Migration8 = Absorption8 + Scat_Out8 - Scat_In8
Fission8 = Xi8 @ Fiss8.T

function8 = Migration8**(-1) @ Fission8

In [98]:
#Group 8, Migration

Migration8

Matrix([
[ 0.0888,       0,       0,       0,       0,       0,       0,       0],
[ -0.053,  0.1193,       0,       0,       0,       0,       0,       0],
[-0.0301, -0.1159,  0.0813,       0,       0,       0,       0,       0],
[-0.0001, -0.0005, -0.0769,  0.2152,       0,       0,       0,       0],
[      0,       0, -0.0019, -0.1961,  0.2529,       0,       0,       0],
[      0,       0,       0,  -0.005, -0.1737,  0.3437, -0.0023,       0],
[      0,       0,       0, -0.0007, -0.0246, -0.2707,   0.417, -0.0275],
[      0,       0,       0, -0.0001, -0.0073,  -0.055, -0.3589,  0.2073]])

In [99]:
#Group 8, Fission

Fission8

Matrix([
[0.00469938, 0.00196392, 0.00038577, 0.00234969, 0.0077154, 0.00778554, 0.03145779, 0.07508487],
[ 0.0055007,  0.0022988, 0.00045155, 0.00275035,  0.009031,  0.0091131, 0.03682185, 0.08788805],
[0.00319992, 0.00133728, 0.00026268, 0.00159996, 0.0052536, 0.00530136, 0.02142036, 0.05112708],
[         0,          0,          0,          0,         0,          0,          0,          0],
[         0,          0,          0,          0,         0,          0,          0,          0],
[         0,          0,          0,          0,         0,          0,          0,          0],
[         0,          0,          0,          0,         0,          0,          0,          0],
[         0,          0,          0,          0,         0,          0,          0,          0]])

In [100]:
#8 Group, Eigenvalues & Eigenvectors

Fun8Eigen = function8.eigenvects()

g = open('NPRE 247_CP 2_output', 'a')
g.write('Eight-Group Eigen:\n')
for i in Fun8Eigen:
    g.write(str(i))
    g.write('\n' + '\n')
g.close()

Fun8Eigen

[(3.02612925575934e-50,
  1,
  [Matrix([
   [       0.265080133597064 + 0.360184012377233*I],
   [ 1.96129509587936e-32 + 2.66495692266183e-32*I],
   [-1.28052588855917e-31 - 1.73994537514172e-31*I],
   [      -0.530160267194128 - 0.720368024754466*I],
   [ 2.59871791272531e-33 + 3.53107051871638e-33*I],
   [-5.31049876175676e-33 - 7.21576802372285e-33*I],
   [ 4.35822600096033e-33 + 5.92184448744394e-33*I],
   [-1.04825937725886e-33 - 1.42434766193032e-33*I]])]),
 (-7.49958435117462e-18 - 4.12813676829274e-50*I,
  1,
  [Matrix([
   [ -0.0869469857479411 + 0.0953589202140202*I],
   [     -0.492347931078626 + 0.5399815378692*I],
   [   -0.414369098813306 + 0.454458420760482*I],
   [   -0.233063655527216 + 0.255612064535935*I],
   [  0.0700240387119938 - 0.0767987143333327*I],
   [  0.0374055913524535 - 0.0410245021222179*I],
   [-0.0066431111033393 + 0.00728581786046797*I],
   [  0.0194513059788459 - 0.0213331781307805*I]])]),
 (-6.33868015224877e-19 - 5.9448806258361e-18*I,
  1,
  [Mat

In [101]:
#8 Group, Power Iteration, Eigenvalue

Max_eigen8 = power_iteration(function8, 2)
Max_eigenvalue8 = Max_eigen8[1]
Max_eigenvalue8

1.09003099735191

In [102]:
#8 Group, Power Iteration, Eigenvector

Max_eigenvector8 = Max_eigen8[0]
Max_eigenvector8 = Matrix(Max_eigenvector8)
Max_eigenvector8

Matrix([
[ 0.28513820857606],
[0.375105720326132],
[0.852381195608686],
[ 0.30559563486994],
[0.243364287345442],
[0.128200066977561],
[0.113916579971508],
[0.239955538334051]])